In [5]:
%pip install langkit[all] -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
os.environ["WHYLABS_DEFAULT_ORG_ID"] = "org-HVB9AM"
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = "model-76"
os.environ["WHYLABS_API_ENDPOINT"] = "https://songbird.development.whylabsdev.com"
os.environ["WHYLABS_API_KEY"] = "z8fYdnQwHr.ibJaqDpZSsZd9dpo5ILyKlOgwXWPV7LGvtIsyqFUs54MGUsHMNz6q"


In [2]:

import pandas as pd
from whylogs.api.writer.whylabs import WhyLabsWriter
from datetime import datetime, timedelta, timezone

import whylogs as why
from langkit import llm_metrics


langkit_schema= llm_metrics.init()

writer = WhyLabsWriter()

starting_date_str = '2023-06-08'
starting_date = datetime.strptime(starting_date_str, '%Y-%m-%d')
starting_date = starting_date.replace(tzinfo = timezone.utc)

In [3]:
from langkit.regexes import has_patterns
from langkit.toxicity import toxicity
from langkit.textstat import automated_readability_index
from langkit.sentiment import sentiment_nltk
from langkit.input_output import similarity_MiniLM_L6_v2
def check_if_ok(pair):
    if has_patterns(pair['response']):
        # print("failed regex for ", pair['response'])
        return False
    if toxicity(pair['prompt']) >= 0.7:
        # print("failed toxicity for ", pair['prompt'])
        return False
    if automated_readability_index(pair['response']) >= 25:
        # print("failed readability for ", pair['response'])
        return False
    if sentiment_nltk(pair['response']) <= -0.7 or sentiment_nltk(pair['prompt']) <= -0.7:
        # print("failed sentiment for ", pair)
        return False
    if similarity_MiniLM_L6_v2(pair) <= 0.3:
        print("failed similarity for ", pair)
        return False
    return True

In [4]:
from datetime import datetime, timedelta, timezone
from typing import Dict
import requests

current_date = datetime.now(timezone.utc)


def download_json(url: str):
    """Download a json file from a url"""
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def get_batch_json_to_pandas():
    batches = {}
    for hour_index in range(48):
        url = f"https://whylabs-public.s3.us-west-2.amazonaws.com/whylogs_examples/LLM-Chatbot-hourly/hourly_batch_{hour_index}.json"
        print(f"Downloading data from {url}")
        single_hour_raw_json = download_json(url)
        filtered_hour_batch = [record for record in single_hour_raw_json if check_if_ok(record)]
        batch_df = pd.DataFrame(filtered_hour_batch)
        batches[hour_index] = batch_df
    return batches

def process_batches(batches: Dict[int, pd.DataFrame]):
    hourly_results = {}
    for hour_index in batches:
        dataset_timestamp = starting_date + timedelta(hours=hour_index, minutes=30)
        dataset_date = dataset_timestamp.replace(tzinfo = timezone.utc)

        hourly_result = why.log(batches[hour_index], schema=langkit_schema)
        print(f"Done profiling batch {hour_index} with whylogs.")
        hourly_result.set_dataset_timestamp(dataset_date)
        hourly_results[hour_index] = hourly_result
    return hourly_results

def inflate_add_url_to_batches(batches):
    processed_batches = {}
    for hour_index in batches:
        url = f"https://whylabs-public.s3.us-west-2.amazonaws.com/whylogs_examples/LLM-Chatbot-hourly/hourly_batch_{hour_index}.json"
        audit_results = why.log({"audit_s3_url": url})
        
        hourly_view = batches[hour_index].merge(audit_results).view()
        for _ in range(3):
            hourly_view = hourly_view.merge(hourly_view)
        print(f"processed profile with timestamp {hourly_view.dataset_timestamp}")
        processed_batches[hour_index] = hourly_view
    return processed_batches



In [5]:
json_batches = get_batch_json_to_pandas()

failed similarity for  {'response': 'Did you do it on purpose?', 'prompt': 'I mistyped a message that made my friend waste time'}
failed similarity for  {'response': 'What is it?', 'prompt': 'I have a confession about the lion king'}
failed similarity for  {'response': "It's good that you look back upon your relationship fondly, but it's over and as your ex has moved on, it's time that you do the same. You deserve to move forward in life and getting over your ex is part of that process. I suggest dating and meeting new people.", 'prompt': "I'm still hoping that he'll come back."}
failed similarity for  {'response': "Really? I hope you are not serious. Are these mixed emotions you're feeling?", 'prompt': "I'm thinking of climbing a rope or poll, last time it caused intense sensations in my pants. Is this too rude to tell you right now?"}
failed similarity for  {'response': 'As the sun began to rise, the sound of chirping birds pierced the stillness of the morning.', 'prompt': 'Generate 

In [6]:
df_batches = process_batches(json_batches)
df_batches

In [18]:
profiles = inflate_add_url_to_batches(df_batches)

processed profile with timestamp 2023-06-08 00:30:00+00:00
processed profile with timestamp 2023-06-08 01:30:00+00:00
processed profile with timestamp 2023-06-08 02:30:00+00:00
processed profile with timestamp 2023-06-08 03:30:00+00:00
processed profile with timestamp 2023-06-08 04:30:00+00:00
processed profile with timestamp 2023-06-08 05:30:00+00:00
processed profile with timestamp 2023-06-08 06:30:00+00:00
processed profile with timestamp 2023-06-08 07:30:00+00:00
processed profile with timestamp 2023-06-08 08:30:00+00:00
processed profile with timestamp 2023-06-08 09:30:00+00:00
processed profile with timestamp 2023-06-08 10:30:00+00:00
processed profile with timestamp 2023-06-08 11:30:00+00:00
processed profile with timestamp 2023-06-08 12:30:00+00:00
processed profile with timestamp 2023-06-08 13:30:00+00:00
processed profile with timestamp 2023-06-08 14:30:00+00:00
processed profile with timestamp 2023-06-08 15:30:00+00:00
processed profile with timestamp 2023-06-08 16:30:00+00:

In [19]:
len(profiles)

48

In [24]:
from time import sleep


for index in profiles:
    if index == 0:
        continue
    view=profiles[index]
    print(f"About to upload profile for time {view.dataset_timestamp}")
    sleep(1)
    status = writer.write(view)
    print(f"uploaded profile for time {view.dataset_timestamp} {status}")
    sleep(1)

About to upload profile for time 2023-06-08 01:30:00+00:00
uploaded profile for time 2023-06-08 01:30:00+00:00 (True, 'log-mcz2s1Lz4UGRUrW0')
About to upload profile for time 2023-06-08 02:30:00+00:00
uploaded profile for time 2023-06-08 02:30:00+00:00 (True, 'log-5ZXRVY1rXGhpTzdz')
About to upload profile for time 2023-06-08 03:30:00+00:00
uploaded profile for time 2023-06-08 03:30:00+00:00 (True, 'log-N72Ar2ZINluCb5Ze')
About to upload profile for time 2023-06-08 04:30:00+00:00
uploaded profile for time 2023-06-08 04:30:00+00:00 (True, 'log-YS2YCloh1Vl5TjeX')
About to upload profile for time 2023-06-08 05:30:00+00:00
uploaded profile for time 2023-06-08 05:30:00+00:00 (True, 'log-udLaOQMO7lvTW91H')
About to upload profile for time 2023-06-08 06:30:00+00:00
uploaded profile for time 2023-06-08 06:30:00+00:00 (True, 'log-DSJFfAIXdj23oDqw')
About to upload profile for time 2023-06-08 07:30:00+00:00
uploaded profile for time 2023-06-08 07:30:00+00:00 (True, 'log-oK1Gsp7s86wf1Euq')
About 

In [27]:
api_key_llm_demo = "5BU9Yfa2kF.vELVWUvT7bfLRQkViiWWmFYsjweblF3qiO8eRnoAzKIEg1sGAgJhX"

In [31]:
os.environ["WHYLABS_API_ENDPOINT"] = "https://api.whylabsapp.com"
prod_writer = WhyLabsWriter(org_id="org-b3VEBV", dataset_id="model-24", api_key=api_key_llm_demo)

for index in profiles:
    view=profiles[index]
    sleep(1)
    status = prod_writer.write(view)
    print(f"About to upload profile for time {view.dataset_timestamp}->{status}")

About to upload profile for time 2023-06-08 00:30:00+00:00->(True, 'log-i7JdQaxulY6BVafx')
About to upload profile for time 2023-06-08 01:30:00+00:00->(True, 'log-NTHllBuu3Z8hqFZ9')
About to upload profile for time 2023-06-08 02:30:00+00:00->(True, 'log-YBd8x0vksyMG9u7W')
About to upload profile for time 2023-06-08 03:30:00+00:00->(True, 'log-HfsgKDbimwPHJGhU')
About to upload profile for time 2023-06-08 04:30:00+00:00->(True, 'log-y63sVHmbk9cwS9uD')
About to upload profile for time 2023-06-08 05:30:00+00:00->(True, 'log-xZCwcDt7yiV6iOn2')
About to upload profile for time 2023-06-08 06:30:00+00:00->(True, 'log-n1XtTeEMIqQlvqBT')
About to upload profile for time 2023-06-08 07:30:00+00:00->(True, 'log-ddMwO3wMsn5Yl5QZ')
About to upload profile for time 2023-06-08 08:30:00+00:00->(True, 'log-IrLwk8qY9z700oKK')
About to upload profile for time 2023-06-08 09:30:00+00:00->(True, 'log-KyUOrxh0LdsEb4aN')
About to upload profile for time 2023-06-08 10:30:00+00:00->(True, 'log-kCHnocVDlVpub4Mi')